## Starting notebook for reading emotibit data

In [2]:
import sys
sys.path.append('../')
from pyeyemo.gaze.eye import Emo
import pyeyemo.gaze.gaze_commons as gm
import pandas as pd
from pathlib import Path
from itertools import compress
import os
import numpy as np
from os import listdir
from os.path import isfile, join,isdir
import matplotlib.pyplot  as plt
import neurokit2 as nk



### Get the list of directories we want to analyse and copy paste the information in sessions.tx



In [7]:
## Get the list of directories we want to analyse and copy paste the information in sessions.tx
session_location = Path(r"P:\Proyectos\2020\PY200016 - XSCAPE\Trabajo\00 XPERIMENTS\01 EXPERIMENTS_Santiago\00 COMPLETED\Exp_Stgo_0001\SESSIONS")
data_path=Path(r"data")
subjects=[]
sessions=[]
window_analysis=6 # total window of analysis from first appearence of label
window_onset=0 # onset time discarded for analysis
annotation_list=['Asset','Surprise','Control'] # list of names that have assets

onlyfiles = [f for f in listdir(session_location) if isdir(join(session_location, f))]
for dir in onlyfiles:
 print (dir,flush=True)

ES0001_S003_PC_EM
ES0001_S004_PC_EM
ES0001_S005_PC_EM
ES0001_S006_PC_EM
ES0001_S008_PC_EM
ES0001_S009_PC_EM
ES0001_S010_PC_EM
ES0001_S011_PC_EM
ES0001_S012_PC_EM
ES0001_S013_PC_EM
ES0001_S015_PC_EM
ES0001_S017_PC_EM
ES0001_S018_PC_EM
ES0001_S019_PC_EM
ES0001_S020_PC_EM
ES0001_S022_PC_EM
ES0001_S023_PC_EM
ES0001_S024_PC_EM
ES0001_S025_PC_EM
ES0001_S026_PC_EM
ES0001_S027_PC_EM
ES0001_S028_PC_EM
ES0001_S029_PC_EM
ES0001_S030_PC_EM
ES0001_S031_PC_EM
Excluded
Resguardo_Excell


### Do the automatic analysis for emotibit data


In [8]:
with open('session.txt','r') as f:
    for session in f:
            sessions.append(session.replace('\n',''))
f.close()
session_loc_list=[session_location.joinpath(session,'00 Raw_data', 'emotibit') for session in sessions ]
# session_loc_list=[session_location.joinpath(session,'01 Processing_data') for session in sessions ]


In [9]:
session_loc_list

[WindowsPath('P:/Proyectos/2020/PY200016 - XSCAPE/Trabajo/00 XPERIMENTS/01 EXPERIMENTS_Santiago/00 COMPLETED/Exp_Stgo_0001/SESSIONS/ES0001_S003_PC_EM/00 Raw_data/emotibit')]

In [10]:
hr_concat=[]
for session, recording_location in zip(sessions,session_loc_list):
     data_paths=gm.extract_session_path_emotibit(recording_location=recording_location,
                                                 subject=session)
     print(session)
     # instantiate an emotibit object
     subject=Emo(data_paths,session)
     # subject.load_heart_rate(data_paths['heart_rate'])
     subject.load_annotations(data_paths['annotations_LM'])
     subject.annotations['LD']=subject.annotations['LD'].str.replace('.tiff','').str.replace('.tif','') #remove .tifs

     subject.labels_to_analyse(annotation_list)
     subject.filter_labels(subject.annotations['LD'])

     subject.label_data_annotation(annotation_time_col='LocalTimestamp',
                                   annotation_col='LD',
                                   label_name='asset',
                                   data_time_col_name='LocalTimestamp')
     subject.skin_cond_resp_amplitude_SA.loc[0,'asset']='blank_0'
     subject.skin_cond_resp_rise_time_SR.loc[0,'asset']='blank_0'
     subject.skin_cond_resp_rise_freq_SF.loc[0,'asset']='blank_0'

     subject.cut_data_of_interest(initial_label=subject.annotations.iloc[np.where(~subject.annotations.LD.isna())].LD.values[0], # get first annotaion value. sometimes in emotibit this fails are there are a few nan values in the amnnotation df
                                  final_label='EndOfExperiment',
                                  filter_column='asset')
     subject.data_z_scores_all_data()
     
     subject.epoch_calculation(myfunction=lambda x: np.mean(x),
                               window_analysis=window_analysis,
                               window_onset=window_onset,
                               data_time_col_name='LocalTimestamp',
                               z_scores=True)

     # subject.heart_rate['diff']=np.insert(np.diff(subject.heart_rate['HR']),0,0)
     
     # signals_HR, info = nk.ppg_process(subject.heart_rate['HR'], sampling_rate=25)
     # subject.heart_rate['ppg']=signals_HR['PPG_Rate'].values
     # subject.heart_rate['PPG_Clean']=signals_HR['PPG_Clean'].values
     
     # subject.data_z_scores(type='HR',
     #                       new_col='hr_z_score',
     #                       col='ppg')
     
     # subject.calculate_hr(window_analysis=window_analysis,
     #                      window_onset=window_onset,
     #                      x_col='hr_z_score')
     
     # subject.heart_rate['asset']=subject.heart_rate['asset'].str.replace('.tiff','').str.replace('.tif','') #remove .tifs
     subjects.append(subject)

     # if len(np.unique(subject.annotations.loc[~subject.annotations_LM['LD'].isna(),'LD']))\
     #      !=len(np.unique(subject.heart_rate_HR.loc[~subject.heart_rate_HR['asset'].isna(),'asset'])):
     #      print(session)
     
     #      print('bad')

# hr_concat=pd.concat([subject.heart_rate_df for subject in subjects ])
# hear_beat_df=pd.concat([subject.heart_beat_inter_beat_BI_df for subject in subjects ])


ES0001_S003_PC_EM


C:\Users\arturoV\AppData\Local\Temp\ipykernel_37268\1480873607.py:10: FutureWarning: The default value of regex will change from True to False in a future version.
  subject.annotations['LD']=subject.annotations['LD'].str.replace('.tiff','').str.replace('.tif','') #remove .tifs


annotations_LM
heart_rate_HR
heart_beat_inter_beat_BI
electrodermal_activity_EA
electrodermal_level_EL
ppg_infrared_PI
ppg_red_PR
ppg_green_PG
accelerometer_x_AX
accelerometer_y_AY
accelerometer_z_AZ
gyroscope_x_GX
gyroscope_y_GY
gyroscope_z_GZ
magnetometer_x_MX
magnetometer_y_MY
magnetometer_z_MZ
skin_cond_resp_amplitude_SA
skin_cond_resp_rise_time_SR
skin_cond_resp_rise_freq_SF
annotations_LM
heart_rate_HR
heart_beat_inter_beat_BI
electrodermal_activity_EA


z:\lab\escritorio\projects\pupil_labs\analysis\pupil_labs_analysis\analysis\emotibit\..\pyeyemo\gaze\eye.py:442: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['std']=self.normalize(values=df[key.rsplit('_')[-1]],


electrodermal_level_EL
ppg_infrared_PI
ppg_red_PR
ppg_green_PG
accelerometer_x_AX
accelerometer_y_AY
accelerometer_z_AZ
gyroscope_x_GX
gyroscope_y_GY
gyroscope_z_GZ
magnetometer_x_MX
magnetometer_y_MY
magnetometer_z_MZ
skin_cond_resp_amplitude_SA
skin_cond_resp_rise_time_SR
skin_cond_resp_rise_freq_SF


In [23]:
hear_beat_df=pd.concat([subject.heart_beat_inter_beat_BI_df for subject in subjects ])
hear_beat_df.Name='hear_beat_df'
electrodermal_activity_df=pd.concat([subject.electrodermal_activity_EA_df for subject in subjects ])
electrodermal_activity_df.Name='electrodermal_activity_df'

electrodermal_level_df=pd.concat([subject.electrodermal_level_EL_df for subject in subjects ])

accelerometer_x_df=pd.concat([subject.accelerometer_x_AX_df for subject in subjects ])
accelerometer_y_df=pd.concat([subject.accelerometer_y_AY_df for subject in subjects ])
accelerometer_z_df=pd.concat([subject.accelerometer_z_AZ_df for subject in subjects ])

gyroscope_x_df=pd.concat([subject.gyroscope_x_GX_df for subject in subjects ])
gyroscope_y_df=pd.concat([subject.gyroscope_y_GY_df for subject in subjects ])
gyroscope_z_df=pd.concat([subject.gyroscope_z_GZ_df for subject in subjects ])

magnetometer_x_df=pd.concat([subject.magnetometer_x_MX_df for subject in subjects ])
magnetometer_y_df=pd.concat([subject.magnetometer_y_MY_df for subject in subjects ])
magnetometer_z_df=pd.concat([subject.magnetometer_z_MZ_df for subject in subjects ])

skin_cond_amp_df=pd.concat([subject.skin_cond_resp_amplitude_SA_df for subject in subjects ])
skin_cond_time_df=pd.concat([subject.skin_cond_resp_rise_time_SR_df for subject in subjects ])
skin_cond_freq_df=pd.concat([subject.skin_cond_resp_rise_freq_SF_df for subject in subjects ])

data_list=[ hear_beat_df,
            electrodermal_activity_df,
            electrodermal_level_df,
            accelerometer_x_df,
            accelerometer_y_df,
            accelerometer_z_df,
            gyroscope_x_df,
            gyroscope_z_df,
            magnetometer_x_df,
            magnetometer_z_df,
            magnetometer_y_df,
            skin_cond_amp_df,
            skin_cond_freq_df,
            skin_cond_time_df ]

In [29]:
data_list[0:1]

[                   Asset0032  Asset0044  Asset0049  Asset0005  Asset0037  \
 ES0001_S003_PC_EM  -0.578942  -0.602032  -0.225673    -0.4101  -0.163331   
 
                    Asset0006  Asset0017  Asset0022  Control01  Surprise02  \
 ES0001_S003_PC_EM    0.62633  -0.501015  -0.080209   0.044475    -0.12177   
 
                    ...  Control04  Asset0039  Asset0027  Asset0003  Asset0011  \
 ES0001_S003_PC_EM  ...  -0.537381  -0.618194  -0.264636    0.30631   -0.12177   
 
                    Asset0021  Asset0029  Asset0012  Asset0007  Asset0046  
 ES0001_S003_PC_EM   1.324557   0.002913   0.127597   3.147704  -0.080209  
 
 [1 rows x 54 columns]]

In [30]:
import seaborn as sns

%matplotlib qt
for index,df in enumerate(data_list[0:1]):
    name=df.Name
    df = df.reindex(sorted(df.columns), axis=1)
    sns.set_theme(style='darkgrid')
    ax, fig=plt.subplots(1,)
    sns.boxplot(data=df).set_title(name)
  


In [27]:
str(data_list[0])

'                   Asset0032  Asset0044  Asset0049  Asset0005  Asset0037  \\\nES0001_S003_PC_EM  -0.578942  -0.602032  -0.225673  -0.410100  -0.163331   \nES0001_S005_PC_EM  -0.329286   0.097374   0.009322  -0.196699  -0.200098   \nES0001_S008_PC_EM   0.113528  -0.093455  -0.468613   0.527496  -0.378058   \nES0001_S010_PC_EM  -0.155902  -0.019744  -0.428217   0.116413   0.071028   \nES0001_S017_PC_EM  -0.245532  -0.387926   1.066122   0.117342  -0.245532   \nES0001_S022_PC_EM  -0.081817  -0.361060  -0.154348  -0.096324  -0.107203   \nES0001_S024_PC_EM  -0.068964  -0.142009   0.077126  -0.068964  -0.434189   \n\n                   Asset0006  Asset0017  Asset0022  Control01  Surprise02  \\\nES0001_S003_PC_EM   0.626330  -0.501015  -0.080209   0.044475   -0.121770   \nES0001_S005_PC_EM  -0.033514  -0.235795   1.465745   3.214880   -0.298689   \nES0001_S008_PC_EM   0.286015   0.286015  -0.196947   0.010037   -0.093455   \nES0001_S010_PC_EM  -0.155902   0.181250  -0.201288  -0.065130   -0.

In [20]:
electrodermal_activity_df.__

<bound method DataFrame.__dataframe__ of                    Asset0032  Asset0044  Asset0049  Asset0005  Asset0037  \
ES0001_S003_PC_EM  -0.589428  -0.698469  -0.951274  -0.753957  -0.474206   
ES0001_S005_PC_EM  -0.492164   0.818009   0.015529   0.305325  -0.815492   
ES0001_S008_PC_EM  -1.427629  -0.070274   0.311191  -0.081207   1.155073   
ES0001_S010_PC_EM   1.413750   0.620942  -0.006584   1.266876  -0.245494   
ES0001_S017_PC_EM   1.400751   0.749306   1.428267   0.650705  -0.047995   
ES0001_S022_PC_EM  -0.740779   0.236533   0.409792  -1.673397  -0.561588   
ES0001_S024_PC_EM   0.792789  -0.750242   0.020785  -0.655809  -1.178261   

                   Asset0006  Asset0017  Asset0022  Control01  Surprise02  \
ES0001_S003_PC_EM  -0.692747  -1.065502  -1.058459  -1.075655   -1.010307   
ES0001_S005_PC_EM  -0.321406  -0.162842  -0.874800  -0.835036   -0.081583   
ES0001_S008_PC_EM  -1.196278  -0.117841   0.645521  -0.514471    0.709413   
ES0001_S010_PC_EM   1.345983   0.487016   

In [21]:
len(subject.gyroscope_x)

gyroscope=subject.gyroscope_x.merge(right=subject.gyroscope_y,on='LocalTimestamp').merge(right=subject.gyroscope_z,on='LocalTimestamp')
gyroscope

,LocalTimestamp,EmotiBitTimestamp_x,PacketNumber_x,DataLength_x,TypeTag_x,ProtocolVersion_x,DataReliability_x,GX,EmotiBitTimestamp_y,PacketNumber_y,...,ProtocolVersion_y,DataReliability_y,GY,EmotiBitTimestamp,PacketNumber,DataLength,TypeTag,ProtocolVersion,DataReliability,GZ
0,1.682067e+09,982760.0,35995,1,GX,1,100,0.671,982760.0,35996,...,1,100,-0.214,982760.0,35997,1,GZ,1,100,0.366
1,1.682067e+09,982795.0,36011,3,GX,1,100,1.129,982795.0,36012,...,1,100,0.427,982795.0,36013,3,GZ,1,100,1.373
2,1.682067e+09,982830.0,36011,3,GX,1,100,-0.397,982830.0,36012,...,1,100,-0.061,982830.0,36013,3,GZ,1,100,0.854
3,1.682067e+09,982865.0,36011,3,GX,1,100,-2.594,982865.0,36012,...,1,100,0.793,982865.0,36013,3,GZ,1,100,0.214
4,1.682067e+09,982905.0,36027,2,GX,1,100,-2.716,982905.0,36028,...,1,100,0.610,982905.0,36029,2,GZ,1,100,-0.061
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30076,1.682068e+09,2177213.0,36112,3,GX,1,100,-13.672,2177213.0,36113,...,1,100,7.721,2177213.0,36114,3,GZ,1,100,19.318
30077,1.682068e+09,2177253.0,36112,3,GX,1,100,-5.493,2177253.0,36113,...,1,100,5.371,2177253.0,36114,3,GZ,1,100,21.088
30078,1.682068e+09,2177293.0,36112,3,GX,1,100,-1.526,2177293.0,36113,...,1,100,0.946,2177293.0,36114,3,GZ,1,100,17.395
30079,1.682068e+09,2177333.0,36126,2,GX,1,100,-1.099,2177333.0,36127,...,1,100,-6.897,2177333.0,36128,2,GZ,1,100,18.341


In [7]:
%matplotlib qt
import seaborn as sns
sns.set_theme(style='darkgrid')
ax, fig=plt.subplots(1,)
sns.boxplot(data=hr_concat)

<AxesSubplot: >

In [46]:
df=pd.read_csv(data_paths['heart_bea_inter_beat'])
df.plot(x='LocalTimestamp',y='BI')

<AxesSubplot: xlabel='LocalTimestamp'>

In [14]:
%matplotlib qt
import seaborn as sns
sns.set_theme(style='darkgrid')
ax, fig=plt.subplots(1,)
sns.boxplot(data= df)

NameError: name 'df' is not defined

In [ ]:
subject.annotations.iloc[np.where(~subject.annotations.LD.isna())].LD.values[0]

In [15]:
a=subject.heart_rate

In [15]:
hr_concat = hr_concat.reindex(sorted(hr_concat.columns), axis=1)
hr_concat

,Asset0001,Asset0002,Asset0003,Asset0004,Asset0005,Asset0006,Asset0007,Asset0008,Asset0009,Asset0010,...,Asset0050,Control01,Control02,Control03,Control04,Control05,Surprise01,Surprise02,Surprise03,Surprise04
ES0001_S003_PC_EM,0.157406,0.555225,0.555225,-0.280195,0.612056,-0.558668,-2.096902,-1.301264,0.290012,-0.737687,...,1.009876,-1.036051,-0.518886,-0.013088,0.157406,-0.638232,1.191736,-0.439323,1.251409,-0.399541
ES0001_S004_PC_EM,0.567182,0.179496,0.709597,0.092465,0.472239,-0.192365,0.567182,-0.667082,0.044993,-0.097422,...,0.614654,-0.619611,0.377295,0.234880,-0.239837,-0.049950,0.614654,0.345647,-0.049950,0.345647
ES0001_S005_PC_EM,-1.128993,-0.672592,-1.282728,-0.048042,0.384338,-0.014413,1.297141,1.245667,0.288254,-0.108095,...,0.456401,-2.306028,-1.743934,-0.720634,0.696613,-0.036032,-3.170789,1.513331,-0.120106,0.024021
ES0001_S006_PC_EM,-0.414827,-0.307938,-0.468272,-0.307938,0.066174,-0.094160,-0.307938,-0.521717,-0.414827,0.012729,...,-0.414827,-0.201049,-0.307938,-0.147605,-0.414827,-0.414827,-0.521717,-0.147605,-0.842384,-0.147605
ES0001_S008_PC_EM,-0.429057,-0.429057,-0.135702,0.255439,-1.406908,-0.722412,-1.504693,1.135505,0.744364,-1.406908,...,1.526645,-0.331272,-0.429057,0.646579,0.646579,-0.624627,0.059868,0.353224,0.451009,0.451009
ES0001_S009_PC_EM,0.120746,-0.521285,-1.002809,-2.246745,0.854497,0.642397,0.120746,-1.939105,1.416274,-0.383707,...,0.361508,-0.340714,0.409661,-1.698343,-2.190567,-1.765221,-0.633641,-1.430830,1.164048,0.201000
ES0001_S010_PC_EM,0.682561,-1.044713,-1.550682,-1.027265,-0.329377,-0.817899,-0.294483,2.008548,2.405472,-1.201737,...,-0.224694,-0.748110,0.019567,-0.678321,-0.259588,-0.050222,-0.538744,-0.154905,0.996610,-1.271526
ES0001_S011_PC_EM,-0.608533,-1.914451,0.247447,0.115758,-0.345155,-0.410999,0.115758,-0.224440,-1.453539,-0.937756,...,-0.015932,1.893562,1.004660,-1.201134,-0.301258,0.510825,-0.147621,-0.608533,0.313291,0.379136
ES0001_S013_PC_EM,-1.034449,-1.560990,-0.771178,0.413539,-0.428927,-0.376273,-0.639543,-0.113002,0.150269,0.676810,...,-0.507908,-0.376273,0.150269,-1.560990,0.281904,1.071716,-0.744851,0.413539,1.334986,-1.034449
ES0001_S017_PC_EM,0.112447,-0.268966,0.632555,-0.442335,0.216469,-0.199618,0.875273,0.043099,1.117990,1.534076,...,-2.106682,0.528534,-0.685052,0.285817,-0.641710,0.008425,0.424512,1.395381,0.216469,-0.615705


In [30]:
subject.heart_rate['ppg']=signals_HR['PPG_Rate'].values
a=subject.heart_rate

In [29]:
signals_HR['PPG_Rate'].values

array([ 58.82352941,  58.82352941,  58.82352941,  58.82352941,
        58.82352941,  58.82352941,  58.82352941,  58.82352941,
        58.82352941,  58.82352941,  58.82352941,  58.82352941,
        58.82352941,  58.82352941,  58.82352941,  58.82352941,
        58.82352941,  58.82352941,  58.82352941,  58.82352941,
        58.82352941,  58.82352941,  58.82352941,  58.82352941,
        58.82352941,  58.82352941,  60.65743675,  62.52399786,
        64.41006515,  66.29962942,  68.1736203 ,  70.00980137,
        71.7828012 ,  73.46432355,  75.02357884,  76.42796967,
        77.64404682,  78.6387262 ,  79.38072395,  79.84212944,
        80.        ,  79.7897775 ,  79.19276568,  78.26583001,
        77.07063352,  75.66949239,  74.12209616,  72.48322148,
        70.80139906,  69.11838791,  67.46926509,  65.8829388 ,
        64.38291882,  62.98821649,  61.71428571,  60.57395143,
        59.57830066,  58.7375331 ,  58.06178587,  57.561961  ,
        57.25059887,  57.14285714,  57.34148589,  57.91

In [15]:
subject.heart_rate['ppg']=signals_HR['PPG_Rate']
a=subject.heart_rate

In [20]:
import neurokit2 as nk
ppg = subject.heart_rate["HR"]
signals_HR, info = nk.ppg_process(ppg, sampling_rate=20)
nk.ppg_plot(signals_HR)


c:\Users\arturoV\AppData\Local\miniconda3\envs\analysis\Lib\site-packages\neurokit2\ppg\ppg_plot.py:67: NeuroKitWarning: 'info' dict not provided. Some information might be missing. Sampling rate will be set to 1000 Hz.
  warn(


ValueError: The data length is too small to be segmented.

In [24]:
signals_HR

,PPG_Raw,PPG_Clean,PPG_Rate,PPG_Peaks
0,72,-0.033756,58.823529,0
1,72,0.108751,58.823529,0
2,72,0.457508,58.823529,0
3,72,0.411926,58.823529,0
4,71,-0.108675,58.823529,0
...,...,...,...,...
641,71,0.189644,60.000000,0
642,71,0.035946,60.000000,0
643,71,0.173483,60.000000,0
644,71,0.094491,60.000000,0


In [25]:
subject.heart_rate


,LocalTimestamp,EmotiBitTimestamp,PacketNumber,DataLength,TypeTag,ProtocolVersion,DataReliability,HR,asset,diff,ppg,PPG_Clean,hr_z_score
311,1.682436e+09,663588,43152,1,HR,1,100,72,blank_0,0,87.124474,-1.426965,0.740660
312,1.682436e+09,664441,43290,1,HR,1,100,72,drift_point_0,0,83.274928,-0.792046,0.593078
313,1.682436e+09,665281,43426,1,HR,1,100,72,drift_point_0,0,79.686865,0.212298,0.455520
314,1.682436e+09,666174,43576,1,HR,1,100,72,drift_point_0,0,76.359695,0.086524,0.327964
315,1.682436e+09,667107,43730,1,HR,1,100,71,Asset0035,-1,73.288141,-0.623251,0.210208
...,...,...,...,...,...,...,...,...,...,...,...,...,...
952,1.682437e+09,1200281,976,1,HR,1,100,71,Control05,-1,NaN,NaN,NaN
953,1.682437e+09,1201134,1110,1,HR,1,100,71,Control05,0,NaN,NaN,NaN
954,1.682437e+09,1202014,1261,1,HR,1,100,71,Control05,0,NaN,NaN,NaN
955,1.682437e+09,1202867,1393,1,HR,1,100,71,Control05,0,NaN,NaN,NaN


In [19]:
hrv_time = nk.hrv_time(signals_HR,sampling_rate=25, show=True)
hrv_time

,HRV_MeanNN,HRV_SDNN,HRV_SDANN1,HRV_SDNNI1,HRV_SDANN2,HRV_SDNNI2,HRV_SDANN5,HRV_SDNNI5,HRV_RMSSD,HRV_SDSD,...,HRV_IQRNN,HRV_SDRMSSD,HRV_Prc20NN,HRV_Prc80NN,HRV_pNN50,HRV_pNN20,HRV_MinNN,HRV_MaxNN,HRV_HTI,HRV_TINN
0,880.888889,425.760401,NaN,NaN,NaN,NaN,NaN,NaN,585.770666,591.511483,...,560.0,0.726838,512.0,1232.0,84.444444,95.555556,400.0,2120.0,9.0,39.0625


In [11]:
hrv_freq = nk.hrv_frequency(signals_HR, sampling_rate=100, show=True, normalize=True)
hrv_freq

NameError: name 'signals_HR' is not defined

In [12]:
signals_HR

NameError: name 'signals_HR' is not defined

In [43]:
hr_concat

,Asset0032,Asset0044,Asset0049,Asset0005,Asset0037,Asset0006,Asset0017,Asset0022,Control01,Surprise02,...,Control04,Asset0039,Asset0027,Asset0003,Asset0011,Asset0021,Asset0029,Asset0012,Asset0007,Asset0046
ES0001_S003_PC_EM,-0.382491,0.754135,0.498394,0.612056,-0.467738,-0.558668,0.214237,-0.865557,-1.036051,-0.439323,...,0.157406,0.100575,0.469978,0.555225,0.072159,-1.036051,-0.373019,1.858217e-01,-2.096902,-2.163206
ES0001_S003_PC_EM,-0.382491,0.754135,0.498394,0.612056,-0.467738,-0.558668,0.214237,-0.865557,-1.036051,-0.439323,...,0.157406,0.100575,0.469978,0.555225,0.072159,-1.036051,-0.373019,1.858217e-01,-2.096902,-2.163206
ES0001_S003_PC_EM,-0.386033,0.752676,0.496466,0.610337,-0.471436,-0.562532,0.211789,-0.869984,-1.040790,-0.442968,...,0.154854,0.097918,0.467998,0.553402,0.069451,-1.040790,-0.376543,1.833214e-01,-2.103584,-2.170009
ES0001_S004_PC_EM,-0.334780,-0.619611,0.139937,0.472239,0.092465,-0.192365,-0.208189,-0.097422,-0.619611,0.345647,...,-0.239837,-0.192365,-0.485108,0.709597,-0.002478,0.994428,-0.382252,-3.822522e-01,0.567182,0.187408
ES0001_S005_PC_EM,0.823707,-1.077908,0.857406,0.390427,-0.211349,-0.009152,0.366356,-0.885339,-2.305532,1.521768,...,0.703351,-0.115065,0.337471,-1.280105,-0.692771,-0.933481,-1.005695,5.290345e-03,1.305128,-0.124693
ES0001_S006_PC_EM,-0.040479,-0.147279,-0.414280,0.066321,-0.147279,-0.093879,-0.467680,-0.254080,-0.200679,-0.147279,...,-0.414280,-0.467680,-0.467680,-0.467680,-0.414280,-0.681280,-0.093879,1.292083e-02,-0.307480,0.279921
ES0001_S003_PC_EM,-0.382491,0.754135,0.498394,0.612056,-0.467738,-0.558668,0.214237,-0.865557,-1.036051,-0.439323,...,0.157406,0.100575,0.469978,0.555225,0.072159,-1.036051,-0.373019,1.858217e-01,-2.096902,-2.163206
ES0001_S003_PC_EM,-0.382491,0.754135,0.498394,0.612056,-0.467738,-0.558668,0.214237,-0.865557,-1.036051,-0.439323,...,0.157406,0.100575,0.469978,0.555225,0.072159,-1.036051,-0.373019,1.858217e-01,-2.096902,-2.163206
ES0001_S004_PC_EM,-0.334780,-0.619611,0.139937,0.472239,0.092465,-0.192365,-0.208189,-0.097422,-0.619611,0.345647,...,-0.239837,-0.192365,-0.485108,0.709597,-0.002478,0.994428,-0.382252,-3.822522e-01,0.567182,0.187408
ES0001_S005_PC_EM,0.816718,-1.080951,0.850348,0.384338,-0.216190,-0.014413,0.360317,-0.888782,-2.306028,1.513331,...,0.696613,-0.120106,0.331492,-1.282728,-0.696613,-0.936824,-1.008887,-9.251859e-18,1.297141,-0.129714


In [42]:
%matplotlib qt
import seaborn as sns
sns.set_theme(style='darkgrid')
ax, fig=plt.subplots(1,)
sns.boxplot(data=hr_concat)

<AxesSubplot: >

In [39]:
%matplotlib qt
import seaborn as sns
sns.set_theme(style='darkgrid')
ax, fig=plt.subplots(1,)
sns.plot()

AttributeError: module 'seaborn' has no attribute 'plot'

In [ ]:
subject.annotation_list

2       Asset0035
5       Control01
8       Asset0047
11      Asset0004
14      Asset0029
17      Asset0015
20      Asset0042
23      Asset0012
26      Asset0032
29     Surprise04
32      Asset0022
35      Asset0008
38      Asset0006
41      Control02
44      Asset0018
47      Asset0010
50      Asset0024
53      Asset0011
56      Asset0049
59      Asset0020
62     Surprise01
65      Asset0037
68      Asset0027
71      Asset0031
74      Asset0034
77      Asset0036
80      Asset0007
83      Asset0041
86      Asset0048
89     Surprise02
92      Asset0028
95      Asset0030
98      Asset0044
101     Asset0039
104     Control04
107     Asset0046
110     Asset0045
113     Asset0050
116     Asset0001
119     Asset0043
122     Asset0033
125    Surprise03
128     Asset0017
131     Asset0040
134     Asset0003
137     Asset0002
140     Asset0014
143     Asset0016
146     Asset0009
149     Control03
152     Asset0005
155     Asset0026
158     Asset0021
161     Control05
163     Asset0008
Name: LD, 

In [ ]:
np.unique(subject.heart_rate.loc[~subject.heart_rate['asset'].isna(),'asset'])

array(['Asset0001', 'Asset0002', 'Asset0003', 'Asset0004', 'Asset0005',
       'Asset0006', 'Asset0007', 'Asset0008', 'Asset0009', 'Asset0010',
       'Asset0011', 'Asset0012', 'Asset0014', 'Asset0015', 'Asset0016',
       'Asset0017', 'Asset0018', 'Asset0020', 'Asset0021', 'Asset0022',
       'Asset0024', 'Asset0026', 'Asset0027', 'Asset0028', 'Asset0029',
       'Asset0030', 'Asset0031', 'Asset0032', 'Asset0033', 'Asset0034',
       'Asset0035', 'Asset0036', 'Asset0037', 'Asset0039', 'Asset0040',
       'Asset0041', 'Asset0042', 'Asset0043', 'Asset0044', 'Asset0045',
       'Asset0046', 'Asset0047', 'Asset0048', 'Asset0049', 'Asset0050',
       'Control01', 'Control02', 'Control03', 'Control04', 'Control05',
       'EndOfExperiment', 'Surprise01', 'Surprise02', 'Surprise03',
       'Surprise04', 'blank_0', 'blank_1', 'blank_10', 'blank_11',
       'blank_12', 'blank_13', 'blank_14', 'blank_15', 'blank_16',
       'blank_17', 'blank_18', 'blank_19', 'blank_2', 'blank_20',
       'blan

In [ ]:
subject.heart_rate.plot(x='EmotiBitTimestamp',y='HR')
subject.heart_rate.plot(x='EmotiBitTimestamp',y='DataReliability')

<AxesSubplot: xlabel='EmotiBitTimestamp'>

In [ ]:
a=subject.annotations['LD']
any(a.str.contains('Asset0031'))
[i for i, x in enumerate(a.str.contains('Asset0031')) if x]

[71]

In [ ]:
a=subject.annotations
len(np.unique(a['LD']))


163

In [ ]:
a['LD']

0              blank_0
1        drift_point_0
2            Asset0035
3              blank_1
4        drift_point_1
            ...       
159           blank_53
160     drift_point_53
161          Control05
162    EndOfExperiment
163          Asset0008
Name: LD, Length: 164, dtype: object

In [ ]:
hr=subject.heart_rate
hr
g=np.unique(hr.loc[~hr['asset'].isna(),'asset'])
len(g)

163

In [ ]:
hr.loc[~hr['asset'].isna(),:]

,LocalTimestamp,EmotiBitTimestamp,PacketNumber,DataLength,TypeTag,ProtocolVersion,DataReliability,HR,asset,hr_z_score
311,1.682436e+09,663588,43152,1,HR,1,100,72,blank_0,-0.143223
312,1.682436e+09,664441,43290,1,HR,1,100,72,drift_point_0,-0.143223
313,1.682436e+09,665281,43426,1,HR,1,100,72,drift_point_0,-0.143223
314,1.682436e+09,666174,43576,1,HR,1,100,72,drift_point_0,-0.143223
315,1.682436e+09,667107,43730,1,HR,1,100,71,Asset0035,-0.512085
...,...,...,...,...,...,...,...,...,...,...
1002,1.682437e+09,1244797,8278,1,HR,1,100,76,Asset0008,1.332226
1003,1.682437e+09,1245836,8446,1,HR,1,100,74,Asset0008,0.594502
1004,1.682437e+09,1246690,8590,1,HR,1,100,73,Asset0008,0.225640
1005,1.682437e+09,1248876,8941,1,HR,1,100,68,Asset0008,-1.618672


In [ ]:
subject['asset']

TypeError: 'Emo' object is not subscriptable

In [ ]:
import pyxdf
data,header=pyxdf.load_xdf(recording_location)


PermissionError: [Errno 13] Permission denied: 'P:\\Proyectos\\2020\\PY200016 - XSCAPE\\Trabajo\\00 XPERIMENTS\\01 EXPERIMENTS_Santiago\\00 COMPLETED\\Exp_Stgo_0001\\SESSIONS\\ES0001_S005_PC_EM\\00 Raw_data\\emotibit'

In [ ]:
data[1]

{'info': defaultdict(list,
             {'name': ['pupil_capture'],
              'type': ['Gaze'],
              'channel_count': ['22'],
              'nominal_srate': ['0'],
              'channel_format': ['double64'],
              'source_id': ['0beb6d28-676c-44f8-8258-c7da93c94a71'],
              'version': ['1.1000000000000001'],
              'created_at': ['193.9634442'],
              'uid': ['6402f7e8-fea5-4363-a0b0-5b612ed067c5'],
              'session_id': ['default'],
              'hostname': ['DESKTOP-3LBQOEJ'],
              'v4address': [None],
              'v4data_port': ['16573'],
              'v4service_port': ['16573'],
              'v6address': [None],
              'v6data_port': ['16573'],
              'v6service_port': ['16573'],
              'desc': [defaultdict(list,
                           {'pupil_lsl_relay_version': ['2.3'],
                            'channels': [defaultdict(list,
                                         {'channel': [defaultdi

In [ ]:
data_sync=data[3]
gaze=data[1]
fixations=data[2]

In [ ]:
data[2]['info']

defaultdict(list,
            {'name': ['pupil_capture_fixations'],
             'type': ['Fixations'],
             'channel_count': ['7'],
             'nominal_srate': ['0'],
             'channel_format': ['double64'],
             'source_id': ['84c04d92-3c63-41ef-ac60-08a23c329d22'],
             'version': ['1.1000000000000001'],
             'created_at': ['9881.3533692000001'],
             'uid': ['73b5e8c2-de44-4d9d-b96d-41a68e4e8d64'],
             'session_id': ['default'],
             'hostname': ['DESKTOP-3LBQOEJ'],
             'v4address': [None],
             'v4data_port': ['16572'],
             'v4service_port': ['16575'],
             'v6address': [None],
             'v6data_port': ['16572'],
             'v6service_port': ['16575'],
             'desc': [defaultdict(list,
                          {'pupil_lsl_relay_version': ['2.3'],
                           'channels': [defaultdict(list,
                                        {'channel': [defaultdict(list,


In [ ]:
data_sync['footer']

{'info': defaultdict(list,
             {'first_timestamp': ['2493.0204354'],
              'last_timestamp': ['3203.3081389'],
              'sample_count': ['164'],
              'clock_offsets': [defaultdict(list,
                           {'offset': [defaultdict(list,
                                         {'time': ['2460.44235175'],
                                          'value': ['-1.175000011244265e-05']}),
                             defaultdict(list,
                                         {'time': ['2465.4548861'],
                                          'value': ['-1.660000020820007e-05']}),
                             defaultdict(list,
                                         {'time': ['2470.46650965'],
                                          'value': ['-1.485000007050985e-05']}),
                             defaultdict(list,
                                         {'time': ['2475.466980499999'],
                                          'value': ['-1.5299999

In [ ]:
a=gaze['time_series']
a

array([[ 1.00000000e+00,  3.34607324e-01,  7.34717941e-01, ...,
         2.92226083e+01,  3.22630099e+00,  3.02459137e+00],
       [ 1.00000000e+00,  3.36215559e-01,  7.36924271e-01, ...,
         2.92226083e+01,  3.21135720e+00,  3.02459137e+00],
       [ 1.00000000e+00,  3.36233116e-01,  7.36243773e-01, ...,
         2.90989313e+01,  3.21135720e+00,  3.01172699e+00],
       ...,
       [ 0.00000000e+00,  6.95890053e-01, -3.34831377e-01, ...,
                    nan,  3.95481640e-03,             nan],
       [ 0.00000000e+00,  6.95890053e-01, -3.34831377e-01, ...,
                    nan,  3.95481640e-03,             nan],
       [ 0.00000000e+00,  6.95890053e-01, -3.34831377e-01, ...,
                    nan,  3.95481640e-03,             nan]])

In [ ]:
b=gaze['time_stamps']
len(b)

185894

In [ ]:
s=fixations['footer']
s

{'info': defaultdict(list,
             {'first_timestamp': ['0'],
              'last_timestamp': ['0'],
              'sample_count': ['0'],
              'clock_offsets': [defaultdict(list,
                           {'offset': [defaultdict(list,
                                         {'time': ['11206.18266655'],
                                          'value': ['-4.849999641010072e-06']}),
                             defaultdict(list,
                                         {'time': ['11211.183215'],
                                          'value': ['-1.759999940986745e-05']}),
                             defaultdict(list,
                                         {'time': ['11216.19787335'],
                                          'value': ['-1.454999983252492e-05']}),
                             defaultdict(list,
                                         {'time': ['11221.20030425'],
                                          'value': ['-9.150000551017001e-06']}),
       

In [ ]:
data_sync['time_stamps']

array([11754.33466454, 11756.32184454, 11758.32239574, 11764.34099274,
       11766.33856824, 11768.33872204, 11774.35554915, 11776.35563135,
       11778.35528185, 11784.37456285, 11786.37191295, 11788.37194085,
       11794.39194666, 11796.38904666, 11798.38881716, 11804.40639416,
       11806.40576046, 11808.40572857, 11814.42259297, 11816.42199357,
       11818.42197807, 11824.43751247, 11826.42224368, 11828.42245678,
       11834.44484008, 11836.43927278, 11838.43874848, 11844.45602239,
       11846.45559219, 11848.45556799, 11854.47262589, 11856.47216889,
       11858.47232999, 11864.4906273 , 11866.4889219 , 11868.4890463 ,
       11874.5070064 , 11876.5055184 , 11878.5055461 , 11884.52515881,
       11886.52324911, 11888.52257711, 11894.53878471, 11896.54032511,
       11898.53892361, 11904.56059962, 11906.55606192, 11908.55558732,
       11914.57549162, 11916.57233622, 11918.57238023, 11924.59170533,
       11926.58897853, 11928.58908993, 11934.60595983, 11936.60584994,
      

In [ ]:
for keys, values in data_sync.items():
    print(keys)


info
footer
time_series
time_stamps


In [ ]:
data_sync['footer']

{'info': defaultdict(list,
             {'first_timestamp': ['11754.3346734'],
              'last_timestamp': ['12329.9632187'],
              'sample_count': ['163'],
              'clock_offsets': [defaultdict(list,
                           {'offset': [defaultdict(list,
                                         {'time': ['11206.1827183'],
                                          'value': ['-6.299999768089037e-06']}),
                             defaultdict(list,
                                         {'time': ['11211.183191'],
                                          'value': ['-7.999999979801942e-06']}),
                             defaultdict(list,
                                         {'time': ['11216.19787465'],
                                          'value': ['-1.115000031859381e-05']}),
                             defaultdict(list,
                                         {'time': ['11221.200296'],
                                          'value': ['-9.200000931

In [ ]:
def lsl_to_df(da:dict):
    pass